# Selenium for 법무법인_태평양_법률자료

### Import Files

In [1]:
import requests
import lxml.html
import sqlite3 as sq3
from pandas.io import sql
import os
import re
import string
import pandas as pd
from tabulate import tabulate
from selenium.webdriver import Chrome
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from selenium import webdriver
from selenium.webdriver import ActionChains
import csv
import numpy as np

## db methods
- reads and writes data from and to db file

In [2]:
def db_save(ARTICLE_LIST, db_name, table_name):
    with sq3.connect(os.path.join('.',db_name)) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            ARTICLE_LIST.to_sql(name = table_name, con = con, index = False, if_exists='replace') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(ARTICLE_LIST), '건 저장완료..')

In [3]:
def db_select(db_name, table_name):
    with sq3.connect(db_name) as con: 
        try:
            query = 'SELECT * FROM {}'.format(table_name)
            df = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e)) 
        return df

## reads the saved file and displays the result as a dataframe

In [4]:
document = db_select('법제동향DB.db','ARTICLE_LIST')

In [5]:
document

,제목,국가,등록일자,조회수,본문내용
0,"카자흐스탄 대통령, 2023년 최저임금 인상",카자흐스탄,2022.09.08.,95,카자흐스탄 입법동향 카자흐스탄 대통령 2023년 최저임금 인상 카심 조마르트 토카...
1,"말레이시아, 개정 고용법 시행 연기",말레이시아,2022.09.05.,342,말레이시아 입법동향 말레이시아 개정 고용법 시행 연기 9월 1일 자로 시행 예정이었...
2,"마카오, 원숭이두창을 《전염병 강제신고》 대상으로 지정",중국 마카오특별행정구,2022.08.01.,887,마카오 법제동향 마카오 원숭이두창을 전염병 강제신고 대상으로 지정 마카오 카지노...
3,형사미성년자에 대한 세계 각국의 연령 기준 및 관련 법령,World,2022.07.22.,1797,형사미성년자에 대한 세계 각국의 연령 기준 및 관련 법령 우리나라 형법제9조에 의하...
4,"독일, 기업조직재편에 관한 유럽연합지침의 국내도입을 위한 법안 결의",독일,2022.07.14.,1194,독일 입법동향 독일 기업조직재편에 관한 유럽연합지침의 국내도입을 위한 법안 결의 ...
...,...,...,...,...,...
5995,이주법 공포,멕시코,2011.05.31.,1836,멕시코입법동향 이주법 공포 2011 0524 펠리페 칼데론 이노호사Felipe C...
5996,노선 택시 정비 방안 채택,우즈베키스탄,2011.05.31.,1652,우즈베키스탄 입법동향 노선 택시 정비 방안 채택 2011년 5월 31일 2012년...
5997,부동산업 면허증제 도입,우즈베키스탄,2011.05.31.,1523,우즈베키스탄 입법동향 부동산업 면허증제 도입 2011년 5월 31일 우즈베키스탄...
5998,벨라루스와 우즈베키스탄 국제 자동차 운송 허가 쿼터 동의,우즈베키스탄,2011.05.31.,1463,우즈베키스탄 입법동향 벨라루스와 우즈베키스탄 국제 자동차 운송 허가 쿼터 동의 ...


In [6]:
document.isna().any()

제목      False
국가      False
등록일자    False
조회수     False
본문내용    False
dtype: bool

## find rows with nan values
edit the table

In [7]:
df_null = document.loc[document.isna().any(axis=1)]
#display(df_null)

In [8]:
print(len(df_null))

0


## remove_punc()
- removes punctuations from string

In [9]:
def remove_punc(data):

    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·「」△《》•‘’○※▷【】『』·-“”)I]'
    new_string = re.sub(punc, '', data) # 특수문자 제거
    new_string = re.sub('\n', ' ', new_string) # newline 제거
    new_string = re.sub('\\s+', ' ', new_string) # multiple spaces 제거
    return new_string